In [1]:
import torch
import torch.nn as nn
import numpy as np

from einops import rearrange

from mosaic_sdf import MosaicSDF
from shape_sampler import ShapeSampler
from optimizer import MosaicSDFOptimizer
from mosaic_sdf_visualizer import MosaicSDFVisualizer

from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

In [2]:

def get_sdf(p):
    
    # print(p.shape)
    s = 1 - np.linalg.norm(p, ord=2, axis=-1)
    # print(s.shape)
    return -s

# model.update_sdf_values(get_sdf)

In [3]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device = 'cpu'
k = 7
n_grids = 1
volume_centers = torch.tensor([
    [0,0,0]
    ], dtype=torch.float32)
volume_scales = torch.tensor([.1], dtype=torch.float32)

cube_wireframe_path = 'data/cube_wireframe.obj'
cow_mesh_path = 'data/cow_mesh/cow.obj'

shape_sampler = ShapeSampler.from_file(cow_mesh_path, sdf_func=get_sdf)

model = MosaicSDF(
    grid_resolution=k,
    n_grids=n_grids,
    volume_centers=volume_centers,
    volume_scales=volume_scales
).to(device)

model.update_sdf_values(shape_sampler)

In [4]:
model.scales

Parameter containing:
tensor([0.1000], requires_grad=True)

In [5]:
# sum of weights should not be 1 when volume is scaled and point is cleary out of grid

model(torch.tensor([
    [.6,0,0]
    ], dtype=torch.float32))

norm sum: tensor([[0.]])
mask: tensor([[True]])


tensor([[1.]], grad_fn=<WhereBackward0>)

In [4]:
model.mosaic_sdf_values

tensor([[[[ 7.3205e-01,  5.6347e-01,  4.5297e-01,  4.1421e-01,  4.5297e-01,
            5.6347e-01,  7.3205e-01],
          [ 5.6347e-01,  3.7437e-01,  2.4722e-01,  2.0185e-01,  2.4722e-01,
            3.7437e-01,  5.6347e-01],
          [ 4.5297e-01,  2.4722e-01,  1.0554e-01,  5.4093e-02,  1.0554e-01,
            2.4722e-01,  4.5297e-01],
          [ 4.1421e-01,  2.0185e-01,  5.4093e-02, -0.0000e+00,  5.4093e-02,
            2.0185e-01,  4.1421e-01],
          [ 4.5297e-01,  2.4722e-01,  1.0554e-01,  5.4093e-02,  1.0554e-01,
            2.4722e-01,  4.5297e-01],
          [ 5.6347e-01,  3.7437e-01,  2.4722e-01,  2.0185e-01,  2.4722e-01,
            3.7437e-01,  5.6347e-01],
          [ 7.3205e-01,  5.6347e-01,  4.5297e-01,  4.1421e-01,  4.5297e-01,
            5.6347e-01,  7.3205e-01]],

         [[ 5.6347e-01,  3.7437e-01,  2.4722e-01,  2.0185e-01,  2.4722e-01,
            3.7437e-01,  5.6347e-01],
          [ 3.7437e-01,  1.5470e-01, -5.9605e-08, -5.7191e-02, -5.9605e-08,
          

In [5]:

visualizer = MosaicSDFVisualizer(model, shape_sampler, 
    device, template_mesh_path=cube_wireframe_path)#, requires_grad=False)

# visualizer.plot_meshes()

/home/che/miniforge3/envs/gen3d/lib/python3.10/site-packages/pytorch3d/io/obj_io.py:544: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


In [ ]:
input_points = torch.tensor([
    [-.5,-.5,0]
    ], dtype=torch.float32)

model(input_points)

In [ ]:
resolution=5
grid_points = torch.stack(torch.meshgrid(
    torch.linspace(-1, 1, resolution),
    torch.linspace(-1, 1, resolution),
    torch.linspace(-1, 1, resolution), indexing='ij'
), dim=-1).reshape(-1, 3)#.to(device)

# print(grid_points.shape)
model(grid_points).reshape(-1, resolution, resolution)

In [ ]:
if True:
    with torch.no_grad():
        meshes = visualizer.create_state_meshes(
            show_mosaic_grids=True,
            show_target_mesh=False,
            resolution=16
            )
        
        # Render the plotly figure
        fig = plot_scene({
            "subplot1": {
                "mesh": meshes
            }
        })
        fig.show()    
    
# Hard boundary on diagonal

In [ ]:
p = torch.tensor([
    [-.99,-.99,-.99],
    [0,0,0],
    [.5,.5,.5]
], dtype=torch.float32)
sdfs = model(p)
sdfs


In [ ]:
# k = 3
# device = 'cpu'

rel_positions = torch.linspace(-1, 1, steps=k, device=device)
grid_coords = torch.stack(
    torch.meshgrid(rel_positions, rel_positions, rel_positions, indexing='ij'), 
    dim=-1).reshape((-1, 3))
# grid_coords

In [ ]:
sdfs = model(grid_coords * .3).reshape((-1, 3))
sdfs


In [ ]:
grid_coords.shape

In [ ]:
n_grids = 6
volume_centers = nn.Parameter(torch.rand((n_grids, 3)) * 2 - 1)  # Initialize randomly within [-1, 1]
print(volume_centers.shape)

scales = nn.Parameter(torch.rand((n_grids,)))
print(scales.shape)